## Week3: IBM Data Capstone Project - Exploring Toronto Neighborhoods

### Submitted by Samy Palaniappan

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

## Importing table from Wiki

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


def data_import(url):
    '''
    function takes in a URL and returns the location of the first table in the webpage
    '''
    postalCode = []
    borough = []
    neighborhood = []
    
    data = requests.get(url).text
    soup = bs(data, 'html.parser')
    
    for row in soup.find('table').find_all('tr'):
        items = row.find_all('td')
        if(len(items)>0):
            postalCode.append(items[0].text.rstrip('\n'))
            borough.append(items[1].text.rstrip('\n'))
            neighborhood.append(items[2].text.rstrip('\n'))
            
    df = pd.DataFrame({"postalCode":postalCode, "borough":borough, "neighborhood":neighborhood})
    return df

df = data_import(url)
df.head(10)

,postalCode,borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


## Exploring the Data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   postalCode    180 non-null    object
 1   borough       180 non-null    object
 2   neighborhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [4]:
df.shape

(180, 3)

In [5]:
df.nunique()

postalCode      180
borough          11
neighborhood    100
dtype: int64

In [6]:
df.isnull().sum()

postalCode      0
borough         0
neighborhood    0
dtype: int64

In [7]:
df.borough.value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: borough, dtype: int64

## Cleaning the Data

### dropping null values for borough

In [8]:
df = df[df.borough!='Not assigned'].reset_index(drop=True)
df.head()

,postalCode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df.shape

(103, 3)

### Grouping boroughs

In [10]:
df_grouped = df.groupby(['borough']).agg(lambda x: ', '.join(x))
df_grouped.head(10)

,postalCode,neighborhood
borough,,
Central Toronto,"M4N, M5N, M4P, M5P, M4R, M5R, M4S, M4T, M4V","Lawrence Park, Roselawn, Davisville North, For..."
Downtown Toronto,"M5A, M7A, M5B, M5C, M5E, M5G, M6G, M5H, M5J, M...","Regent Park, Harbourfront, Queen's Park, Ontar..."
East Toronto,"M4E, M4K, M4L, M4M, M7Y","The Beaches, The Danforth West, Riverdale, Ind..."
East York,"M4B, M4C, M4G, M4H, M4J","Parkview Hill, Woodbine Gardens, Woodbine Heig..."
Etobicoke,"M9A, M9B, M9C, M9P, M9R, M8V, M9V, M8W, M9W, M...","Islington Avenue, Humber Valley Village, West ..."
Mississauga,M7R,Canada Post Gateway Processing Centre
North York,"M3A, M4A, M6A, M3B, M6B, M3C, M2H, M3H, M2J, M...","Parkwoods, Victoria Village, Lawrence Manor, L..."
Scarborough,"M1B, M1C, M1E, M1G, M1H, M1J, M1K, M1L, M1M, M...","Malvern, Rouge, Rouge Hill, Port Union, Highla..."
West Toronto,"M6H, M6J, M6K, M6P, M6R, M6S","Dufferin, Dovercourt Village, Little Portugal,..."


In [11]:
df_grouped.shape

(10, 2)

### setting neighborhood with 'Not assigned' tag to its corresponding borough tab

In [12]:
for row in range(len(df)):
    if df.loc[row, 'neighborhood'] =='Not assigned':
        df.loc[row, 'neighborhood'] == df.loc[row, 'borough']

In [13]:
df_neigh = df[df.neighborhood=='Not assigned']
df_neigh.shape

(0, 3)

In [14]:
df.shape

(103, 3)

In [15]:
print('The number of datapoints with uniqe postal_codes after cleaning is ', df.shape[0])

The number of datapoints with uniqe postal_codes after cleaning is  103


## Part-II

###  Adding latitude longitudes to corresponding postal codes.

In [16]:
lat_long_df = pd.read_csv('Geospatial_Coordinates.csv')
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
full_df = df.merge(right=lat_long_df, how='left', left_on='postalCode', right_on='Postal Code')
full_df = full_df.drop(columns='Postal Code')
full_df.head()

,postalCode,borough,neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part-III

### Map Visualization with Folium

In [18]:
from geopy.geocoders import Nominatim

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(full_df['Latitude'], full_df['Longitude'], full_df['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare Data

In [26]:
client_id = '2EK22ZCG0TPU5RGWSLHGNAHMOGFNKALDPVF5S3ASZPLDV1N3'
client_secret = 'RUTLTPU3BFRBVS5LUL0S1JYYWWYT1KIPC5MY4DD3WMKQYXUM'

In [63]:
CLIENT_ID = client_id # your Foursquare ID
CLIENT_SECRET = client_secret # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2EK22ZCG0TPU5RGWSLHGNAHMOGFNKALDPVF5S3ASZPLDV1N3
CLIENT_SECRET:RUTLTPU3BFRBVS5LUL0S1JYYWWYT1KIPC5MY4DD3WMKQYXUM


In [64]:
full_df.borough.unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [65]:
borough_list = list(full_df.borough.unique())
tor_list = []
for i in borough_list:
    if 'Toronto' in i:
        tor_list.append(i)
tor_list

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [66]:
tor_df = full_df[full_df.borough.isin(tor_list)].reset_index(drop=True)
tor_df.head()

,postalCode,borough,neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [67]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [81]:
LIMIT = 10
radius=500
venues_list = []
for lat, long, postcode, borough, neighborhood in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['postalCode'], tor_df['borough'], tor_df['neighborhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
    url # display URL
    results = requests.get(url).json()["response"]['groups'][0]['items']
    # return only relevant information for each nearby venue
    venues_list.append([(
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.321558,Leslie St. Spit hotdog stand,43.652255,-79.322791,Hot Dog Joint
1,"Regent Park, Harbourfront",43.654260,-79.321558,TTC Leslie Barns,43.657633,-79.325212,Light Rail Station
2,"Queen's Park, Ontario Provincial Government",43.662301,-79.321558,Rorschach Brewing Co.,43.663483,-79.319824,Brewery
3,"Queen's Park, Ontario Provincial Government",43.662301,-79.321558,Leslieville Farmers Market,43.664901,-79.319784,Farmers Market
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.321558,The Sidekick,43.664484,-79.325162,Comic Shop


### Analyze each Neighborhood§

In [84]:
# one hot encoding
tor_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Neighborhood,Asian Restaurant,Auto Workshop,Bar,Boat or Ferry,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,...,Playground,Restaurant,Sandwich Place,Scenic Lookout,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Sushi Restaurant
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Queen's Park, Ontario Provincial Government",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Queen's Park, Ontario Provincial Government",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Queen's Park, Ontario Provincial Government",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
tor_onehot.shape

(199, 46)

In [98]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Asian Restaurant,Auto Workshop,Bar,Boat or Ferry,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,...,Playground,Restaurant,Sandwich Place,Scenic Lookout,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Sushi Restaurant
0,Berczy Park,0.0,0.000000,0.0,0.500000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.0,0.333333,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"Business reply mail Processing Centre, South C...",0.0,0.100000,0.0,0.000000,0.0,0.0,0.100000,0.0,0.1,...,0.000000,0.1,0.0,0.000000,0.1,0.000000,0.0,0.0,0.000000,0.0
3,Central Bay Street,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Christie,0.1,0.000000,0.1,0.000000,0.0,0.0,0.100000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.1,0.0,0.000000,0.0
5,Church and Wellesley,0.0,0.000000,0.0,0.000000,0.0,0.0,0.100000,0.0,0.1,...,0.000000,0.1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.1
6,"Commerce Court, Victoria Hotel",0.0,0.000000,0.0,0.333333,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
7,Davisville North,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.333333,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.333333,0.0
8,"Dufferin, Dovercourt Village",0.1,0.000000,0.1,0.000000,0.0,0.0,0.100000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.1
9,"First Canadian Place, Underground city",0.0,0.000000,0.0,0.333333,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0


### Top venues of each neighborhood

In [99]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
              venue  freq
0   Harbor / Marina   0.5
1     Boat or Ferry   0.5
2  Asian Restaurant   0.0
3       Pizza Place   0.0
4     Hot Dog Joint   0.0


----Brockton, Parkdale Village, Exhibition Place----
              venue  freq
0     Boat or Ferry  0.33
1    Scenic Lookout  0.33
2              Park  0.33
3  Asian Restaurant  0.00
4          Pharmacy  0.00


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0  Garden Center   0.1
1  Auto Workshop   0.1
2        Brewery   0.1
3     Skate Park   0.1
4  Burrito Place   0.1


----Central Bay Street----
                venue  freq
0  Light Rail Station   1.0
1    Asian Restaurant   0.0
2           Gastropub   0.0
3     Harbor / Marina   0.0
4       Hot Dog Joint   0.0


----Christie----
               venue  freq
0  Indian Restaurant   0.3
1   Asian Restaurant   0.1
2                Bar   0.1
3        Snack Place   0.1
4            Brewery   0.1




In [100]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Boat or Ferry,Harbor / Marina,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop
1,"Brockton, Parkdale Village, Exhibition Place",Park,Boat or Ferry,Scenic Lookout,Sushi Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
2,"Business reply mail Processing Centre, South C...",Garden Center,Restaurant,Farmers Market,Comic Shop,Pizza Place,Brewery,Burrito Place,Fast Food Restaurant,Skate Park,Auto Workshop
3,Central Bay Street,Light Rail Station,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
4,Christie,Indian Restaurant,Asian Restaurant,Snack Place,Bar,Brewery,Fish & Chips Shop,Café,Pakistani Restaurant,Cocktail Bar,Farmers Market


### Clustering

In [138]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 4, 1, 1, 3, 1, 1, 3], dtype=int32)

In [139]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
venues_sorted.head()
# tor_merge = venues_sorted

# # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# tor_merge = tor_merge.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# tor_merge.head() # check the last columns!



,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Berczy Park,Boat or Ferry,Harbor / Marina,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop
1,1,"Brockton, Parkdale Village, Exhibition Place",Park,Boat or Ferry,Scenic Lookout,Sushi Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
2,1,"Business reply mail Processing Centre, South C...",Garden Center,Restaurant,Farmers Market,Comic Shop,Pizza Place,Brewery,Burrito Place,Fast Food Restaurant,Skate Park,Auto Workshop
3,4,Central Bay Street,Light Rail Station,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
4,1,Christie,Indian Restaurant,Asian Restaurant,Snack Place,Bar,Brewery,Fish & Chips Shop,Café,Pakistani Restaurant,Cocktail Bar,Farmers Market


In [140]:
venues_sorted[venues_sorted['Cluster Labels']==1]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,1,"Brockton, Parkdale Village, Exhibition Place",Park,Boat or Ferry,Scenic Lookout,Sushi Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
2,1,"Business reply mail Processing Centre, South C...",Garden Center,Restaurant,Farmers Market,Comic Shop,Pizza Place,Brewery,Burrito Place,Fast Food Restaurant,Skate Park,Auto Workshop
4,1,Christie,Indian Restaurant,Asian Restaurant,Snack Place,Bar,Brewery,Fish & Chips Shop,Café,Pakistani Restaurant,Cocktail Bar,Farmers Market
5,1,Church and Wellesley,Fast Food Restaurant,Brewery,Farmers Market,Comic Shop,Fish & Chips Shop,Pizza Place,Restaurant,Sushi Restaurant,Burrito Place,Breakfast Spot
7,1,Davisville North,Soccer Stadium,Skating Rink,Playground,Sushi Restaurant,Bus Stop,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop
8,1,"Dufferin, Dovercourt Village",Indian Restaurant,Sushi Restaurant,Pakistani Restaurant,Bar,Brewery,Café,Farmers Market,Fish & Chips Shop,Asian Restaurant,Playground
10,1,"Forest Hill North & West, Forest Hill Road Park",Pharmacy,Bus Line,Skating Rink,Dance Studio,Bus Stop,Farmers Market,Dog Run,Cosmetics Shop,Comic Shop,Coffee Shop
13,1,"High Park, The Junction South",Garden Center,Restaurant,Farmers Market,Comic Shop,Pizza Place,Brewery,Burrito Place,Fast Food Restaurant,Skate Park,Auto Workshop
14,1,"India Bazaar, The Beaches West",Indian Restaurant,Sushi Restaurant,Pakistani Restaurant,Bar,Brewery,Café,Farmers Market,Fish & Chips Shop,Asian Restaurant,Playground
16,1,Lawrence Park,French Restaurant,Grocery Store,Coffee Shop,Park,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop


In [141]:
venues_sorted[venues_sorted['Cluster Labels']==2]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,2,"Kensington Market, Chinatown, Grange Park",Hot Dog Joint,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
22,2,"Regent Park, Harbourfront",Hot Dog Joint,Light Rail Station,Sushi Restaurant,Café,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
23,2,"Richmond, Adelaide, King",Hot Dog Joint,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
26,2,"Runnymede, Swansea",Hot Dog Joint,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
27,2,St. James Town,Hot Dog Joint,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop


In [142]:
venues_sorted[venues_sorted['Cluster Labels']==3]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Berczy Park,Boat or Ferry,Harbor / Marina,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop
6,3,"Commerce Court, Victoria Hotel",Boat or Ferry,Harbor / Marina,Hot Dog Joint,Sushi Restaurant,Café,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop
9,3,"First Canadian Place, Underground city",Boat or Ferry,Harbor / Marina,Hot Dog Joint,Sushi Restaurant,Café,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop
12,3,"Harbourfront East, Union Station, Toronto Islands",Boat or Ferry,Harbor / Marina,Scenic Lookout,Sushi Restaurant,Café,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop
17,3,"Little Portugal, Trinity",Boat or Ferry,Hot Dog Joint,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop
20,3,"Parkdale, Roncesvalles",Harbor / Marina,Hot Dog Joint,Sushi Restaurant,Café,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop


In [143]:
venues_sorted[venues_sorted['Cluster Labels']==4]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,4,Central Bay Street,Light Rail Station,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
11,4,"Garden District, Ryerson",Light Rail Station,Sushi Restaurant,Café,Fast Food Restaurant,Farmers Market,Dog Run,Dance Studio,Cosmetics Shop,Comic Shop,Coffee Shop
